In [1]:
# More data in the Amazon Cloud in a PostgreSQL RDS

library(RPostgreSQL)

conn <- dbConnect(drv=PostgreSQL(), 
                  host="pg-ais-inst.chf8mgp53zo6.us-west-2.rds.amazonaws.com", port=5432,
                  user="rockper", password="1InTen-10", dbname="ais")

Loading required package: DBI


In [3]:
# Two tables in AWS: 1) Years 2015, 2016 & 2017 for Zone 11 only (Puget Sound), and 2) All UTM zones for Dec 2017

dbListTables(conn)

[1] "minloc"   "allzones"

In [4]:
# data <- dbGetQuery(con, "SELECT * from sales")

query <- "SELECT base_dt AT TIME ZONE 'UTC' AT TIME ZONE 'PST' AS datetime, date_part('dow', base_dt) AS DOW, mmsi, lat, lon, sog, cog 
FROM allzones 
WHERE vessel_type=35 AND lat > 32.5 AND lat < 34 AND lon > -120.0 AND lon < -114.0 AND sog > 10 
ORDER BY datetime ASC 
LIMIT 10;"

dbGetQuery(conn, query)

datetime,dow,mmsi,lat,lon,sog,cog
2017-12-05 08:29:39,2,367384340,32.66213,-118.0129,12.3,85.9
2017-12-05 08:30:43,2,367384340,32.66228,-118.0087,12.2,87.4
2017-12-05 08:31:48,2,367384340,32.66246,-118.0043,12.3,87.5
2017-12-05 08:32:53,2,367384340,32.66266,-118.0001,12.2,84.7
2017-12-05 08:33:56,2,367384340,32.66282,-117.9959,12.0,87.1
2017-12-05 08:34:59,2,367384340,32.66296,-117.9918,12.0,87.7
2017-12-05 08:36:00,2,367384340,32.66299,-117.9877,12.2,91.5
2017-12-05 08:37:10,2,367384340,32.66299,-117.9832,12.0,92.0
2017-12-05 08:38:16,2,367384340,32.66306,-117.9787,11.6,87.8
2017-12-05 08:39:23,2,367384340,32.66330,-117.9744,12.1,86.4


In [5]:
dbDisconnect(conn)

[1] TRUE

In [2]:
# Processing
library(dplyr)
library(stringr)
library(data.table)
library(tidyr)
library(magrittr) # for the pipes %>%
library(ggplot2)

# Mapping
library(maps)
library(ggmap)
library(mapdata)
library(rgdal)
library(broom)
library(maptools)
library(sf)  # will use GEOS 3.6.2, GDAL 2.2.3 and proj 4.4.9.3


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last


Attaching package: ‘magrittr’

The following object is masked from ‘package:tidyr’:

    extract

Google Maps API Terms of Service: http://developers.google.com/maps/terms.
Please cite ggmap if you use it: see citation("ggmap") for details.

Attaching package: ‘ggmap’

The following object is masked from ‘package:magrittr’:

    inset

Loading required package: sp
rgdal: version: 1.2-20, (SVN revision 725)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.2.3, released 2017/11/20
 Path to GDAL shared files: /usr/share/gdal/2.2
 GDAL binary built with GEOS: TRUE 
 Loaded PROJ.4 runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION:

In [29]:
# Examine activity in UTM Zone 18 in December 2017
# North Carolina to Connecticut, Longitude -78 to -72

# Linux
# setwd("/media/rock/x/data/ais_20180921/allzones_2017_12/")

# Windows
setwd("D:/data/ais/allzones_2017_12/")
fName <- "AIS_2017_12_Zone18.csv"

In [30]:
# Read in the CSV dataset

colNames <- c("mmsi", "dt_string", "lat", "lon", "sog", "cog", "heading", "vessel_name", "imo", "call_sign",
             "vessel_type", "status", "len", "width", "draft", "cargo")
dTypes <- c("factor", "character", "numeric", "numeric", "numeric", "numeric", "numeric", "character", 
            "character", "character", "factor", "character", "numeric", "numeric", "numeric", "integer")

df <- read.csv(fName, header=TRUE, sep=",", col.names=colNames, colClasses = dTypes, stringsAsFactors=FALSE)

# Convert the datetime string to POSIXct format in the UTC timezone
df$base_dt <- strptime(df$dt_string, "%Y-%m-%dT%H:%M:%S", tz = "GMT")

In [68]:
# Function to create a ViewFrame nMiles N,S,E & W of a vessel at a given latitude and longitude

getViewFrame <- function (nMiles, lat, lon) {
    latMax <- lat + nMiles/69.172
    latMin <- lat - nMiles/69.172
    lonMax <- lon + nMiles/(cos(lat)*69.172)
    lonMin <- lon - nMiles/(cos(lat)*69.172)
    return(c(latMin, latMax, lonMin, lonMax))
}

In [32]:
# Finds the Military Vessels (vessel_type = 35)

militaryVesselsDf <- df[df$vessel_type==35,]
militaryVesselMMSIs <- unique(militaryVesselsDf$mmsi)

In [39]:
library(plyr)
plyr::count(militaryVesselsDf, 'mmsi')
# mmsi:count 368926164:1580, 369970634:2164

------------------------------------------------------------------------------
You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)
------------------------------------------------------------------------------

Attaching package: ‘plyr’

The following object is masked from ‘package:maps’:

    ozone

The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize



mmsi,freq
111480697,261
368926164,1580
368926165,760
368926171,52
369970634,2164


In [45]:
# Track one of five military vessels in Zone 18

mv2Df <- militaryVesselsDf[militaryVesselsDf$mmsi==369970634, c("base_dt", "lat", "lon", "sog", "cog")]
mv2Df <- mv2Df[order(mv2Df$base_dt),]

In [55]:
mv2MotionDf <- mv2Df[mv2Df$sog > 10,]
# mv2MotionDf

In [58]:
write.csv(mv2MotionDf, file="mv2.csv", row.names=FALSE)

In [64]:
mv2Motion10MinuteDf <- mv2MotionDf[seq(1, 100, 10), ]

In [65]:
mv2Motion10MinuteDf

,base_dt,lat,lon,sog,cog
9156062,2017-12-11 14:54:58,36.97686,-76.32861,10.4,46.4
9071163,2017-12-11 15:06:28,36.99961,-76.29575,10.9,58.9
8504924,2017-12-11 15:17:42,37.00708,-76.24277,15.6,92.9
9218928,2017-12-11 15:28:55,36.99235,-76.18446,15.0,108.8
9264413,2017-12-11 15:39:56,36.97945,-76.13654,10.3,110.5
9191380,2017-12-11 15:51:10,36.96686,-76.08350,18.5,108.0
21004898,2017-12-11 16:02:05,36.94960,-76.01617,19.2,108.9
9247993,2017-12-11 16:12:54,36.92902,-75.96245,12.0,121.3
9127664,2017-12-11 16:24:24,36.90453,-75.92679,11.5,138.3
9262651,2017-12-11 16:36:04,36.87767,-75.89745,10.5,138.7


In [72]:
for (i in 1:nrow(mv2Motion10MinuteDf)) {
    t <- mv2Motion10MinuteDf$base_dt[i]
    latmv <- mv2Motion10MinuteDf$lat[i]
    lonmv <- mv2Motion10MinuteDf$lon[i]
    frame <- getViewFrame(4, latmv, lonmv)
    viewFrameDf <- subset(df, (base_dt < t) & (base_dt >= t-60) & 
                          (lat > frame[1]) & (lat < frame[2]) & 
                          (lon > frame[3]) & (lon < frame[4]))
    filename <- paste0("position", i, ".csv")
    write.csv(viewFrameDf, filename, row.names=FALSE)
}